In [ ]:
# 빈 데이터 프레임 생성

insert_df = pd.DataFrame()
n_counts = 0

# df_channel - 채널id 데이터 프레임

for channel_id in df_channel['channel_id']:
    try:
        video_url = "https://www.youtube.com/channel/{}/community".format(channel_id)
        session = HTMLSession()
        response = session.get(video_url)
        n_counts += 1
        if(response.status_code == 429):
            print(response)
        soup = bs(response.html.html, "html.parser")
        data = re.search(r"var ytInitialData = ({.*?});", soup.prettify()).group(1)
        data_json = json.loads(data)
        commu = data_json['contents']['twoColumnBrowseResultsRenderer']['tabs'][3]['tabRenderer']['content']['sectionListRenderer']['contents'][0]['itemSectionRenderer']['contents']
        text = []
        day = []
        for i in range(0, len(commu)-2):
            if len(commu[i]['backstagePostThreadRenderer']['post']['backstagePostRenderer']['contentText']['runs']) == 1:
                text.append(commu[i]['backstagePostThreadRenderer']['post']['backstagePostRenderer']['contentText']['runs'][0]['text'])
                day.append(commu[i]['backstagePostThreadRenderer']['post']['backstagePostRenderer']['publishedTimeText']['runs'][0]['text'])
            else:
                k = ''
                for j in range(len(commu[i]['backstagePostThreadRenderer']['post']['backstagePostRenderer']['contentText']['runs'])):
                    k += commu[i]['backstagePostThreadRenderer']['post']['backstagePostRenderer']['contentText']['runs'][j]['text']  
                text.append(k)
                day.append(commu[i]['backstagePostThreadRenderer']['post']['backstagePostRenderer']['publishedTimeText']['runs'][0]['text'])
                
        df = pd.DataFrame() # 커뮤니티 텍스트 담을 데이터 프레임
        df['community'] = text # 커뮤니티 텍스트
        df['channel_id'] = channel_id # 채널 id
        df['date'] = day # 시간
        insert_df = pd.concat([insert_df,df]) # 기존에 데이터 프레임에 붙여가면서 저장
    except:
        continue